In [1]:
import pickle as pkl
from copy import deepcopy
import jsonlines
from utils.my_util import cluster_mentions, remove_speaker_prefix
import json

In [2]:
original_data = {}
with open('data/raw_source/dialogue_zh/all_coref_data_en.json', 'r') as f:
    temp = json.load(f)
    for line in temp:
        scene_id = line['scene_id']
        original_data[scene_id] = line

In [3]:
projected_data = {}
with open('data/raw_source/dialogue_fa/all_coref_data_en_fa.json', 'r') as f:
    temp = json.load(f)
    for line in temp:
        scene_id = line['scene_id']
        projected_data[scene_id] = line

In [4]:
print(original_data.keys())

dict_keys(['s01e01c00t0', 's01e01c01t0', 's01e01c03t0', 's01e01c04t0', 's01e01c05t0', 's01e01c06t0', 's01e01c07t0', 's01e01c08t0', 's01e01c09t0', 's01e01c10t0', 's01e03c00t0', 's01e10c04t0', 's01e03c01t0', 's01e03c02t0', 's01e03c03t0', 's01e03c04t0', 's01e03c05t0', 's01e03c06t0', 's01e03c07t0', 's01e03c08t0', 's01e03c09t0', 's01e04c00t0', 's01e04c01t0', 's01e04c02t0', 's01e04c03t0', 's01e04c04t0', 's01e04c05t0', 's01e04c06t0', 's01e04c07t0', 's01e04c08t0', 's01e04c09t0', 's01e04c10t0', 's01e04c11t0', 's01e04c12t0', 's01e05c00t0', 's01e05c01t0', 's01e05c02t0', 's01e05c03t0', 's01e05c04t0', 's01e05c05t0', 's01e05c06t0', 's01e05c07t0', 's01e07c01t0', 's01e07c02t0', 's01e07c03t0', 's01e07c04t0', 's01e07c05t0', 's01e09c00t0', 's01e09c01t0', 's01e09c02t0', 's01e09c03t0', 's01e09c04t0', 's01e09c05t0', 's01e09c06t0', 's01e07c00t0', 's01e09c07t0', 's01e10c00t0', 's01e10c01t0', 's01e10c02t0', 's01e10c03t0', 's01e10c05t0', 's01e10c07t0', 's01e11c00t0', 's01e11c01t0', 's01e11c02t0', 's01e11c03t0',

In [5]:
print(projected_data.keys())

dict_keys(['s01e01c00t0', 's01e01c01t0', 's01e01c03t0', 's01e01c04t0', 's01e01c05t0', 's01e01c06t0', 's01e01c07t0', 's01e01c08t0', 's01e01c09t0', 's01e01c10t0', 's01e03c00t0', 's01e10c04t0', 's01e03c01t0', 's01e03c02t0', 's01e03c03t0', 's01e03c04t0', 's01e03c05t0', 's01e03c06t0', 's01e03c07t0', 's01e03c08t0', 's01e03c09t0', 's01e04c00t0', 's01e04c01t0', 's01e04c02t0', 's01e04c03t0', 's01e04c04t0', 's01e04c05t0', 's01e04c06t0', 's01e04c07t0', 's01e04c08t0', 's01e04c09t0', 's01e04c10t0', 's01e04c11t0', 's01e04c12t0', 's01e05c00t0', 's01e05c01t0', 's01e05c02t0', 's01e05c03t0', '', 's01e05c05t0', 's01e05c06t0', 's01e05c07t0', 's01e07c01t0', 's01e07c02t0', 's01e07c03t0', 's01e07c04t0', 's01e07c05t0', 's01e09c00t0', 's01e09c01t0', 's01e09c02t0', 's01e09c03t0', 's01e09c04t0', 's01e09c05t0', 's01e09c06t0', 's01e07c00t0', 's01e09c07t0', 's01e10c00t0', 's01e10c01t0', 's01e10c02t0', 's01e10c03t0', 's01e10c05t0', 's01e10c07t0', 's01e11c00t0', 's01e11c01t0', 's01e11c02t0', 's01e11c03t0', 's01e11c04

In [9]:
for i in list(projected_data.keys())[:10]:
    if projected_data[i]['scene_id']=="":
        continue
    sample = projected_data[i]
    sentences = sample['sentences']
    # sentences = [[token for token in "".join(sent)] for sent in sample['sentences']]
    # print(sentences)
    print(sample['scene_id'])
    for answer in sample['annotations']:
        query = answer['query']
        antecedents = answer['antecedents']
        print(query, sentences[query['sentenceIndex']][query['startToken']: query['endToken']])
        print(antecedents)
        print(sentences[query['sentenceIndex']], len(sentences[query['sentenceIndex']]))
        print()

s01e01c00t0
{'sentenceIndex': 0, 'startToken': 3, 'endToken': 10} ['به', 'طرف', 'يک', 'صفحه', 'اي', 'که', 'داراي']
['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']
['اگه', 'يه', 'فوتون', 'به', 'طرف', 'يک', 'صفحه', 'اي', 'که', 'داراي', 'دو', 'شياره', 'هدايت', 'بشه', 'و', 'هردو', 'شيار', 'تحت', 'نظر', 'باشن', 'اين', 'فوتون', 'همزمان', 'از', 'هردو', 'شيار', 'رد', 'نميشه', 'اگه', 'تحت', 'نظر', 'باشه', '،', 'رد', 'خواهد', 'شد', 'اما', 'اگر', 'اين', 'فوتون', 'پس', 'از', 'عبور', 'از', 'صفحه', 'و', 'قبل', 'از', 'رسيدن', 'به', 'هدفش', 'تحت', 'نظر', 'باشه', 'هرگز', 'همزمان', 'از', 'هر', 'دوشيار', 'عبور', 'نميکنه'] 61

{'sentenceIndex': 0, 'startToken': 15, 'endToken': 21} ['هردو', 'شيار', 'تحت', 'نظر', 'باشن', 'اين']
['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']
['اگه', 'يه', 'فوتون', 'به', 'طرف', 'يک', 'صفحه', 'اي', 'که', 'داراي', 'دو', 'شياره', 'هدايت', 'بشه', 'و', 'هردو', 'شيار', 'تحت', 'نظر', 'باشن', 'اين', 'فوتون', 'همزمان', 'از', 'هردو', 'شيار', 'رد', 'نميشه', 'اگه', 'تحت', 'نظر', '

In [13]:
for i in list(projected_data.keys())[:10]:
    if projected_data[i]['scene_id']=="":
        continue
    original = original_data[i]
    projected = projected_data[i]
    sentence = projected['sentences']
    print(original['scene_id'])
    for answer in projected['annotations']:
        query = answer['query']
        antecedents = answer['antecedents']
        if isinstance(antecedents[0], str):
            continue
        print("Sentence:", sentence[query['sentenceIndex']])
        # print("".join(sentence[query['sentenceIndex']]))
        # print("".join(sentence[query['sentenceIndex']])[query['startToken']: query['endToken']])
        print("Query:", query, sentence[query['sentenceIndex']][query['startToken']: query['endToken']])
        # print("Query:", query, sentence[query['sentenceIndex']][query['startToken']: query['endToken']])
        print("Maximum Sentence Index:", len(sentence[query['sentenceIndex']]))
        print()
    print("=="*50)

s01e01c00t0
Sentence: ['اگه', 'يه', 'فوتون', 'به', 'طرف', 'يک', 'صفحه', 'اي', 'که', 'داراي', 'دو', 'شياره', 'هدايت', 'بشه', 'و', 'هردو', 'شيار', 'تحت', 'نظر', 'باشن', 'اين', 'فوتون', 'همزمان', 'از', 'هردو', 'شيار', 'رد', 'نميشه', 'اگه', 'تحت', 'نظر', 'باشه', '،', 'رد', 'خواهد', 'شد', 'اما', 'اگر', 'اين', 'فوتون', 'پس', 'از', 'عبور', 'از', 'صفحه', 'و', 'قبل', 'از', 'رسيدن', 'به', 'هدفش', 'تحت', 'نظر', 'باشه', 'هرگز', 'همزمان', 'از', 'هر', 'دوشيار', 'عبور', 'نميکنه']
Query: {'sentenceIndex': 0, 'startToken': 46, 'endToken': 50} ['قبل', 'از', 'رسيدن', 'به']
Maximum Sentence Index: 61

Sentence: ['اگه', 'يه', 'فوتون', 'به', 'طرف', 'يک', 'صفحه', 'اي', 'که', 'داراي', 'دو', 'شياره', 'هدايت', 'بشه', 'و', 'هردو', 'شيار', 'تحت', 'نظر', 'باشن', 'اين', 'فوتون', 'همزمان', 'از', 'هردو', 'شيار', 'رد', 'نميشه', 'اگه', 'تحت', 'نظر', 'باشه', '،', 'رد', 'خواهد', 'شد', 'اما', 'اگر', 'اين', 'فوتون', 'پس', 'از', 'عبور', 'از', 'صفحه', 'و', 'قبل', 'از', 'رسيدن', 'به', 'هدفش', 'تحت', 'نظر', 'باشه', 'هرگز', 'هم

In [42]:
sample = "将光子正对平面上的双缝观察任意一个隙缝它不会穿过那两个隙缝如果没被观察那就会总之如果观察它在离开平面到击中目标之前它就不会穿过那两个隙缝"
print([token for token in sample])

['将', '光', '子', '正', '对', '平', '面', '上', '的', '双', '缝', '观', '察', '任', '意', '一', '个', '隙', '缝', '它', '不', '会', '穿', '过', '那', '两', '个', '隙', '缝', '如', '果', '没', '被', '观', '察', '那', '就', '会', '总', '之', '如', '果', '观', '察', '它', '在', '离', '开', '平', '面', '到', '击', '中', '目', '标', '之', '前', '它', '就', '不', '会', '穿', '过', '那', '两', '个', '隙', '缝']
